# Credit Risk Resampling Techniques

In [9]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [11]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [13]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

loan_amnt  int_rate  installment home_ownership  annual_inc  \
0    10500.0    0.1719       375.35           RENT     66000.0   
1    25000.0    0.2000       929.09       MORTGAGE    105000.0   
2    20000.0    0.2000       529.88       MORTGAGE     56000.0   
3    10000.0    0.1640       353.55           RENT     92000.0   
4    22000.0    0.1474       520.39       MORTGAGE     52000.0   

  verification_status   issue_d loan_status pymnt_plan    dti  ...  \
0     Source Verified  Mar-2019    low_risk          n  27.24  ...   
1            Verified  Mar-2019    low_risk          n  20.23  ...   
2            Verified  Mar-2019    low_risk          n  24.26  ...   
3            Verified  Mar-2019    low_risk          n  31.44  ...   
4        Not Verified  Mar-2019    low_risk          n  18.76  ...   

   pct_tl_nvr_dlq  percent_bc_gt_75  pub_rec_bankruptcies  tax_liens  \
0            85.7             100.0                   0.0        0.0   
1            91.2              50.0                   1.0        0.0   
2            66.7              50.0                   0.0        0.0   
3           100.0              50.0                   1.0        0.0   
4           100.0               0.0                   0.0        0.0   

   tot_hi_cred_lim  total_bal_ex_mort total_bc_limit  \
0          65687.0            38199.0         2000.0   
1         271427.0            60641.0        41200.0   
2          60644.0            45684.0         7500.0   
3          99506.0            68784.0        19700.0   
4         219750.0            25919.0        27600.0   

   total_il_high_credit_limit  hardship_flag  debt_settlement_flag  
0                     61987.0              N                     N  
1                     49197.0              N                     N  
2                     43144.0              N                     N  
3                     76506.0              N                     N  
4                     20000.0              N                     N  

[5 rows x 86 columns]

In [14]:

df_encoded = pd.get_dummies(df, columns=["hardship_flag", "debt_settlement_flag", "application_type", "issue_d", "next_pymnt_d", "initial_list_status", "pymnt_plan", "verification_status", "home_ownership"])
df_encoded.head()

loan_amnt  int_rate  installment  annual_inc loan_status    dti  \
0    10500.0    0.1719       375.35     66000.0    low_risk  27.24   
1    25000.0    0.2000       929.09    105000.0    low_risk  20.23   
2    20000.0    0.2000       529.88     56000.0    low_risk  24.26   
3    10000.0    0.1640       353.55     92000.0    low_risk  31.44   
4    22000.0    0.1474       520.39     52000.0    low_risk  18.76   

   delinq_2yrs  inq_last_6mths  open_acc  pub_rec  ...  initial_list_status_f  \
0          0.0             0.0       8.0      0.0  ...                      0   
1          0.0             0.0      17.0      1.0  ...                      0   
2          0.0             0.0       8.0      0.0  ...                      0   
3          0.0             1.0      10.0      1.0  ...                      0   
4          0.0             1.0      14.0      0.0  ...                      0   

   initial_list_status_w  pymnt_plan_n  verification_status_Not Verified  \
0                      1             1                                 0   
1                      1             1                                 0   
2                      1             1                                 0   
3                      1             1                                 0   
4                      1             1                                 1   

   verification_status_Source Verified  verification_status_Verified  \
0                                    1                             0   
1                                    0                             1   
2                                    0                             1   
3                                    0                             1   
4                                    0                             0   

   home_ownership_ANY  home_ownership_MORTGAGE  home_ownership_OWN  \
0                   0                        0                   0   
1                   0                        1                   0   
2                   0                        1                   0   
3                   0                        0                   0   
4                   0                        1                   0   

   home_ownership_RENT  
0                    1  
1                    0  
2                    0  
3                    1  
4                    0  

[5 rows x 96 columns]

# Split the Data into Training and Testing

In [15]:
list(df_encoded.columns.values)

['loan_amnt',
 'int_rate',
 'installment',
 'annual_inc',
 'loan_status',
 'dti',
 'delinq_2yrs',
 'inq_last_6mths',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'total_acc',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_amnt',
 'collections_12_mths_ex_med',
 'policy_code',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'open_acc_6m',
 'open_act_il',
 'open_il_12m',
 'open_il_24m',
 'mths_since_rcnt_il',
 'total_bal_il',
 'il_util',
 'open_rv_12m',
 'open_rv_24m',
 'max_bal_bc',
 'all_util',
 'total_rev_hi_lim',
 'inq_fi',
 'total_cu_tl',
 'inq_last_12m',
 'acc_open_past_24mths',
 'avg_cur_bal',
 'bc_open_to_buy',
 'bc_util',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'mo_sin_old_il_acct',
 'mo_sin_old_rev_tl_op',
 'mo_sin_rcnt_rev_tl_op',
 'mo_sin_rcnt_tl',
 'mort_acc',
 'mths_since_recent_bc',
 'mths_since_recent_inq',
 'num_accts_ever_120_pd'

In [18]:
# Create our features
y = df_encoded["loan_status"]

# Create our target
x = df_encoded.drop(columns="loan_status")

In [19]:
x.describe()

loan_amnt      int_rate   installment    annual_inc           dti  \
count  68817.000000  68817.000000  68817.000000  6.881700e+04  68817.000000   
mean   16677.594562      0.127718    480.652863  8.821371e+04     21.778153   
std    10277.348590      0.048130    288.062432  1.155800e+05     20.199244   
min     1000.000000      0.060000     30.890000  4.000000e+01      0.000000   
25%     9000.000000      0.088100    265.730000  5.000000e+04     13.890000   
50%    15000.000000      0.118000    404.560000  7.300000e+04     19.760000   
75%    24000.000000      0.155700    648.100000  1.040000e+05     26.660000   
max    40000.000000      0.308400   1676.230000  8.797500e+06    999.000000   

        delinq_2yrs  inq_last_6mths      open_acc       pub_rec  \
count  68817.000000    68817.000000  68817.000000  68817.000000   
mean       0.217766        0.497697     12.587340      0.126030   
std        0.718367        0.758122      6.022869      0.336797   
min        0.000000        0.000000      2.000000      0.000000   
25%        0.000000        0.000000      8.000000      0.000000   
50%        0.000000        0.000000     11.000000      0.000000   
75%        0.000000        1.000000     16.000000      0.000000   
max       18.000000        5.000000     72.000000      4.000000   

           revol_bal  ...  initial_list_status_f  initial_list_status_w  \
count   68817.000000  ...           68817.000000           68817.000000   
mean    17604.142828  ...               0.123879               0.876121   
std     21835.880400  ...               0.329446               0.329446   
min         0.000000  ...               0.000000               0.000000   
25%      6293.000000  ...               0.000000               1.000000   
50%     12068.000000  ...               0.000000               1.000000   
75%     21735.000000  ...               0.000000               1.000000   
max    587191.000000  ...               1.000000               1.000000   

       pymnt_plan_n  verification_status_Not Verified  \
count       68817.0                      68817.000000   
mean            1.0                          0.478007   
std             0.0                          0.499520   
min             1.0                          0.000000   
25%             1.0                          0.000000   
50%             1.0                          0.000000   
75%             1.0                          1.000000   
max             1.0                          1.000000   

       verification_status_Source Verified  verification_status_Verified  \
count                         68817.000000                  68817.000000   
mean                              0.373992                      0.148001   
std                               0.483865                      0.355104   
min                               0.000000                      0.000000   
25%                               0.000000                      0.000000   
50%                               0.000000                      0.000000   
75%                               1.000000                      0.000000   
max                               1.000000                      1.000000   

       home_ownership_ANY  home_ownership_MORTGAGE  home_ownership_OWN  \
count        68817.000000             68817.000000        68817.000000   
mean             0.009285                 0.526309            0.106747   
std              0.095914                 0.499311            0.308793   
min              0.000000                 0.000000            0.000000   
25%              0.000000                 0.000000            0.000000   
50%              0.000000                 1.000000            0.000000   
75%              0.000000                 1.000000            0.000000   
max              1.000000                 1.000000            1.000000   

       home_ownership_RENT  
count         68817.000000  
mean              0.357659  
std               0.479314  
min               0.000000  
25%    

In [20]:
y.describe()

count        68817
unique           2
top       low_risk
freq         68470
Name: loan_status, dtype: object

In [21]:
y.head()

0    low_risk
1    low_risk
2    low_risk
3    low_risk
4    low_risk
Name: loan_status, dtype: object

In [23]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [27]:
# Create X_train, X_test, y_train, y_test
# YOUR CODE HERE

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1)
Counter(y_train)

Counter({'low_risk': 51366, 'high_risk': 246})

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [28]:
# Resample the training data with the RandomOversampler
# YOUR CODE HERE

from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
x_resampled, y_resampled = ros.fit_resample(x_train, y_train)
Counter(y_resampled)

Counter({'low_risk': 51366, 'high_risk': 51366})

In [30]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(x_resampled, y_resampled)

LogisticRegression(random_state=1)

In [32]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.6664317419814947

In [33]:
# Display the confusion matrix
# YOUR CODE HERE

from sklearn.metrics import confusion_matrix
y_pred = model.predict(x_test)
confusion_matrix(y_test, y_pred)

array([[   73,    28],
       [ 6669, 10435]])

In [34]:
# Print the imbalanced classification report
# YOUR CODE HERE

from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.72      0.61      0.02      0.66      0.45       101
   low_risk       1.00      0.61      0.72      0.76      0.66      0.44     17104

avg / total       0.99      0.61      0.72      0.75      0.66      0.44     17205



### SMOTE Oversampling

In [35]:
# Resample the training data with SMOTE
# YOUR CODE HERE

from imblearn.over_sampling import SMOTE
x_resampled, y_resampled = SMOTE(random_state=1,
sampling_strategy='auto').fit_resample(
   x_train, y_train)
Counter(y_resampled)

Counter({'low_risk': 51366, 'high_risk': 51366})

In [37]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(x_resampled, y_resampled)

LogisticRegression(random_state=1)

In [38]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

y_pred = model.predict(x_test)
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.6601038260982319

In [39]:
# Display the confusion matrix
# YOUR CODE HERE

from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[   63,    38],
       [ 5192, 11912]])

In [40]:
# Print the imbalanced classification report
# YOUR CODE HERE

from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.62      0.70      0.02      0.66      0.43       101
   low_risk       1.00      0.70      0.62      0.82      0.66      0.44     17104

avg / total       0.99      0.70      0.62      0.82      0.66      0.44     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [41]:
# Resample the data using the ClusterCentroids resampler
# YOUR CODE HERE

from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
x_resampled, y_resampled = cc.fit_resample(x_train, y_train)
Counter(y_resampled)

Counter({'high_risk': 246, 'low_risk': 246})

In [42]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(x_resampled, y_resampled)

LogisticRegression(random_state=1)

In [43]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

y_pred = model.predict(x_test)
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.544392082449592

In [44]:
# Display the confusion matrix
# YOUR CODE HERE

from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  68,   33],
       [9997, 7107]])

In [45]:
# Print the imbalanced classification report
# YOUR CODE HERE

from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.67      0.42      0.01      0.53      0.29       101
   low_risk       1.00      0.42      0.67      0.59      0.53      0.27     17104

avg / total       0.99      0.42      0.67      0.58      0.53      0.27     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [46]:
# Resample the training data with SMOTEENN
# YOUR CODE HERE

from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=0)
x_resampled, y_resampled = smote_enn.fit_resample(x, y)

In [47]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(x_resampled, y_resampled)

LogisticRegression(random_state=1)

In [48]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

y_pred = model.predict(x_test)
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.6483657924959942

In [49]:
# Display the confusion matrix
# YOUR CODE HERE

from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  73,   28],
       [7287, 9817]])

In [50]:
# Print the imbalanced classification report
# YOUR CODE HERE

from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.72      0.57      0.02      0.64      0.42       101
   low_risk       1.00      0.57      0.72      0.73      0.64      0.41     17104

avg / total       0.99      0.57      0.72      0.72      0.64      0.41     17205

